## Syntax-dependent trace



In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import os
CACHE_DIR = "/scratch/lucchetti.f/models/"
os.environ['TRANSFORMERS_CACHE'] = CACHE_DIR


import sys
sys.path.append('..')

import gc
import torch
from model_utils import check_devs, clear_devs
import tqdm as notebook_tqdm
from model import ModelLoader

# from model_loader import ModelLoader
# from model_manager import ModelManager
# from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig

#MODEL_NAME = "Salesforce/codegen-16B-mono"
MODEL_NAME = "bigcode/santacoder"

check_devs()

/home/lucchetti.f/.local/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0 / 34079899648 used for device 0, reserved 0


In [ ]:
lm = ModelLoader(MODEL_NAME)
lm.model

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


In [ ]:
check_devs()

## Inference

In [ ]:
model = lm.model
tokenizer = lm.tokenizer

prompts = ["""# intialize three integers
           int x = 1
           int y = 2
           int z ="""]

prompts = ["""int x ="""]

# vanilla generate:

tokenized = tokenizer(prompts, return_tensors="pt")
outputs = model.generate(tokenized.input_ids.to(model.device), max_new_tokens=2, top_k = 1)
tokenizer.decode(outputs[0])

## Trace + activation patch

In [ ]:
txt, ret_dict = mt.generate(
    prompts,
    argmax_greedy= True,
    debug = True,
    request_activations= [mt.layer_name_format.format(i) for i in range(5,10)]
)
txt